In [ ]:
import json
import csv

def process_file(input_file, output_file):
    with open(input_file, 'r') as f_in, open(output_file, 'w', newline='') as f_out:
        writer = csv.writer(f_out)
        # Write the header row
        writer.writerow(['date_added', 'rating', 'book_id'])
        
        for i, line in enumerate(f_in):
            if (i%1000000 == 0):
                print(i/1000000)
            try:
                record = json.loads(line)
            except json.JSONDecodeError:
                # Skip lines that are not valid JSON
                continue
            
            # Check if required keys exist and are not null
            if (record.get('date_added') is not None and
                record.get('rating') is not None and
                record.get('book_id') is not None):
                writer.writerow([record['date_added'], record['rating'], record['book_id']])

# Example usage:
if __name__ == '__main__':
    process_file('datasets/goodreads_reviews_dedup.json', 'outcomes/output.csv')


0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0


In [ ]:
import json
import pandas as pd

data = []
output_file = "datasets/reviews_parquet"
input_file = "datasets/goodreads_reviews_dedup.json"
#input_file = "small2.json"


In [3]:

with open(input_file, 'r') as f_in:
    for i, line in enumerate(f_in):
        if i % 1_000_000 == 0:
            print(f"Processed {i / 1_000_000}M lines...")

        try:
            record = json.loads(line)
        except json.JSONDecodeError:
            continue  # Skip invalid JSON lines
        
        # Check if required keys exist and are not null
        if all(key in record and record[key] is not None for key in ['date_added', 'rating', 'title']):
            data.append([record['date_added'], record['rating'], record['title']])

    # Convert to DataFrame and write to Parquet
    df = pd.DataFrame(data, columns=['date_added', 'rating', 'title'])
    df.to_parquet(output_file, engine='pyarrow', compression='snappy', index=False)

Processed 0.0M lines...
Processed 1.0M lines...
Processed 2.0M lines...
Processed 3.0M lines...
Processed 4.0M lines...
Processed 5.0M lines...
Processed 6.0M lines...
Processed 7.0M lines...
Processed 8.0M lines...
Processed 9.0M lines...
Processed 10.0M lines...
Processed 11.0M lines...
Processed 12.0M lines...
Processed 13.0M lines...
Processed 14.0M lines...
Processed 15.0M lines...


In [4]:
import pandas as pd
df = pd.read_parquet(output_file)
print(df.head())
display(df)


Empty DataFrame
Columns: [date_added, rating, title]
Index: []


,date_added,rating,title


In [ ]:
def comp_months(df):
    split_date = df['date_added'].str.split()
    print("split")
    df['year'] = pd.to_numeric(split_date.str[-1], errors='coerce', downcast='integer')
    print("yeared")

    # Extract the second element as the month and map it to numbers
    month_mapping = {
        'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
        'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
    }
    df['month'] = split_date.str[1].map(month_mapping)

    print("mapped")

    # Compute months since December 2006
    df['month'] = ((df['year'] - 2006) * 12) + (df['month'] - 12)
    print("monthed")

    # Drop intermediate year column
    return df.drop(columns=['year'])

    # Save the updated DataFrame back to Parquet

#import dask.dataframe as dd
#ddf = dd.read_parquet(output_file, engine='pyarrow')
#print("read old")
#ddf = ddf.map_partitions(comp_months)
#ddf.to_parquet("reviews_parquet_monthed.parquet", engine='pyarrow', compression='snappy', write_index=False)
df = comp_months(df)




split
yeared
mapped
monthed


In [11]:
display(df)
df = df[df['month'] >= 0]
smallest_month_row = df.loc[df['month'].idxmin()]
print(smallest_month_row)



,date_added,rating,book_id,month
0,Fri Aug 25 13:55:02 -0700 2017,5,24375664,128
1,Sun Jul 30 07:44:10 -0700 2017,5,18245960,127
2,Mon Jul 24 02:48:17 -0700 2017,3,6392944,127
3,Mon Jul 24 02:33:09 -0700 2017,4,22078596,127
4,Mon Jul 24 02:28:14 -0700 2017,4,6644782,127
...,...,...,...,...
15739962,Mon Jun 04 18:08:44 -0700 2012,4,1656001,66
15739963,Fri Aug 01 18:46:18 -0700 2014,5,10024429,92
15739964,Tue Aug 27 12:49:25 -0700 2013,3,6721437,80
15739965,Fri May 03 13:06:15 -0700 2013,2,15788197,77


date_added    Fri Dec 29 10:59:10 -0800 2006
rating                                     5
book_id                                 9624
month                                      0
Name: 221, dtype: object


In [13]:
#ddf = ddf.drop(columns=["month_num"])
df = df.drop(columns=['date_added'])
display(df)



,rating,book_id,month
0,5,24375664,128
1,5,18245960,127
2,3,6392944,127
3,4,22078596,127
4,4,6644782,127
...,...,...,...
15739962,4,1656001,66
15739963,5,10024429,92
15739964,3,6721437,80
15739965,2,15788197,77


In [ ]:
df.to_parquet("datasets/reviews_parquet_monthed.parquet", engine='pyarrow', compression='snappy')

In [ ]:
df = pd.read_parquet("datasets/reviews_parquet_monthed.parquet")


,rating,book_id,month
0,5,24375664,128
1,5,18245960,127
2,3,6392944,127
3,4,22078596,127
4,4,6644782,127
...,...,...,...
15739962,4,1656001,66
15739963,5,10024429,92
15739964,3,6721437,80
15739965,2,15788197,77


In [21]:
import math
display(math.log2(int(df["book_id"].max())))

23.253495942863836

In [23]:
df = df.astype({
    "rating": "int8",
    "month": "uint8",
    "book_id": "int32"
})
display(df)

,rating,book_id,month
0,5,24375664,128
1,5,18245960,127
2,3,6392944,127
3,4,22078596,127
4,4,6644782,127
...,...,...,...
15739962,4,1656001,66
15739963,5,10024429,92
15739964,3,6721437,80
15739965,2,15788197,77


In [ ]:
import pandas as pd
import json

# Filepath for the input JSONL file
input_file = "datasets/goodreads_books.json"  # Change this to the correct file path
output_file = "datasets/metadata.parquet"

# List to store relevant data
data = []

# Read the JSONL file line by line and extract required fields
with open(input_file, "r", encoding="utf-8") as f:
    for line in f:
        entry = json.loads(line.strip())
        data.append({
            "isbn": int(entry["isbn"]) if entry["isbn"].isdigit() else None,
            "title": entry["title"],
            "book_id": int(entry["book_id"]),
            "isbn13": int(entry["isbn13"]) if entry["isbn13"].isdigit() else None
        })

# Create DataFrame
df = pd.DataFrame(data)

# Set book_id as index and convert datatypes
df.set_index("book_id", inplace=True)

# Save to Parquet
df.to_parquet(output_file, engine="pyarrow")



In [26]:
display(df)

,isbn,title,isbn13
book_id,,,
5333265,312853122.0,W.C. Fields: A Life on Film,9.780313e+12
1333909,743509986.0,Good Harbor,9.780744e+12
7327624,NaN,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",NaN
6066819,743294297.0,Best Friends Forever,9.780743e+12
287140,850308712.0,Runic Astrology: Starcraft and Timekeeping in ...,9.780850e+12
...,...,...,...
3084038,563553014.0,"This Sceptred Isle, Vol. 10: The Age of Victor...",9.780564e+12
26168430,NaN,Sherlock Holmes and the July Crisis,9.781781e+12
2342551,765197456.0,The Children's Classic Poetry Collection,9.780765e+12
